In [ ]:
import cv2
import numpy as np
from face_action import FaceAction
import csv
import os
import matplotlib.pyplot as plt


In [ ]:
def process_frame(frame, brightness, contrast, hist_eq, canny_ed, detect_fatigue_enabled, dilation_enabled, erosion_enabled, kernel_size, face_action):
    processed_frame = frame.copy()

    processed_frame = apply_grayscale(processed_frame)
    processed_frame = apply_blur(processed_frame)
    processed_frame = apply_brightness(processed_frame, brightness)
    processed_frame = apply_contrast(processed_frame, contrast)
    processed_frame = apply_hist_eq(processed_frame, hist_eq)
    processed_frame = apply_canny_edge_detection(processed_frame, canny_ed)
    processed_frame = apply_dilation_erosion(processed_frame, dilation_enabled, erosion_enabled, kernel_size)
    processed_frame = apply_detect_fatigue(processed_frame, detect_fatigue_enabled, face_action)

    return processed_frame

In [ ]:
def apply_grayscale(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return frame


In [ ]:
def apply_blur(frame):
    frame = cv2.GaussianBlur(frame, (5, 5), 0)
    return frame


In [ ]:
def apply_brightness(frame, brightness):
    frame = cv2.add(frame, brightness)
    frame = np.clip(frame, 0, 255)
    return frame


In [ ]:
def apply_contrast(frame, contrast):
    frame = cv2.multiply(frame, contrast)
    frame = np.clip(frame, 0, 255)
    return frame


In [ ]:
def apply_hist_eq(frame, hist_eq):
    if hist_eq:
        channels = cv2.split(frame)
        equalized_channels = [cv2.equalizeHist(channel) for channel in channels]
        frame = cv2.merge(equalized_channels)
    return frame


In [ ]:
def apply_detect_fatigue(frame, detect_fatigue_enabled, face_action):
    person_data_ = []
    if detect_fatigue_enabled:
        results = face_action.run_frame(frame)
        fatigue_count = 0
        for i, (ear, mar, leftEye, rightEye, mouth, leftEAR, rightEAR) in enumerate(results, start=1):
            person_data = [
                f"Personne:{i} ",
                f"{leftEAR:.2f} ",
                f"{rightEAR:.2f} ",
                f"{ear:.2f} ",
                f"{mar:.2f} "
            ]
            for point in rightEye:
                cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
            for point in leftEye:
                cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
            for point in mouth:
                cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
            cv2.putText(frame, f"œil gauche : {leftEAR:.2f}", (leftEye[0][0] - 190, leftEye[0][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            cv2.putText(frame, f"œil droite : {rightEAR:.2f}", (leftEye[0][0] + 40, leftEye[0][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
            if ear < 0.25 or mar > 0.7:
                fatigue_count += 1
            person_data_.append(person_data)
            print(person_data_)
            save_person_data(person_data_)
    
    return frame

In [ ]:
def apply_canny_edge_detection(frame, canny_ed):
    if canny_ed:
        if frame.dtype != np.uint8:
            frame = frame.astype(np.uint8)
        frame = cv2.Canny(frame, 50, 150)
    return frame


In [ ]:
def apply_dilation_erosion(frame, dilation_enabled, erosion_enabled, kernel_size):
    if dilation_enabled:
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        frame = cv2.dilate(frame, kernel, iterations=1)

    if erosion_enabled:
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        frame = cv2.erode(frame, kernel, iterations=1)

    return frame


In [ ]:
def save_person_data(last_detected_person_data):
    filename = "person_data_note.csv"
    write_header = not os.path.exists(filename)

    with open(filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        if write_header:
            csv_writer.writerow(["Person", "Left Ear", "Right Ear", "EAR", "MAR"])

        if last_detected_person_data:
            csv_writer.writerows(last_detected_person_data)


In [ ]:
cap = cv2.VideoCapture(0)
face_action = FaceAction()

brightness = 0
contrast = 1
hist_eq = False
canny_ed = False
detect_fatigue_enabled = True
dilation_enabled = False
erosion_enabled = False
kernel_size = 3

ret, frame = cap.read()
cap.release()
processed_frame = process_frame(frame, brightness, contrast, hist_eq, canny_ed, detect_fatigue_enabled, dilation_enabled, erosion_enabled, kernel_size, face_action)

# Display the processed frame using Matplotlib
plt.imshow(cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB))
plt.show()